In [1]:
import numpy as np
from models.base_models import Transformer
from arithmetic_sampler import ArithmeticSampler
from config import get_config
from train import train
import torch
import torch.nn as nn
import torch.nn.functional as F
import utils as u

%load_ext autoreload
%autoreload 2

In [34]:
config = get_config()
sampler = ArithmeticSampler(config.task.max_variables, config.task.max_seq_len)
torch.cuda.empty_cache()

In [16]:
%%timeit
sampler.generate(2**17)

2.39 s ± 49.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
model = Transformer(config)

train(model, sampler, config, verbose=False)

Results are saved in:  results\train_bbfad07a08bd3de0de55daf02c6bd160


Button(description='Stop Training', style=ButtonStyle())

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
Transformer                              [128, 82]                 [128, 82, 16]             --
├─Embedding: 1-1                         [128, 82]                 [128, 82, 64]             1,024
├─ModuleList: 1-2                        --                        --                        --
│    └─TFBlock: 2-1                      [128, 82, 64]             [128, 82, 64]             --
│    │    └─LayerNorm: 3-1               [128, 82, 64]             [128, 82, 64]             128
│    │    └─MultiHeadAttention: 3-2      [128, 82, 64]             [128, 82, 64]             16,384
│    │    └─Dropout: 3-3                 [128, 82, 64]             [128, 82, 64]             --
│    │    └─LayerNorm: 3-4               [128, 82, 64]             [128, 82, 64]             128
│    │    └─Sequential: 3-5              [128, 82, 64]             [128, 82, 64]             33,088
│    │    └─Dropout: 3

eval/accuracy,▁▁▅▆▇▇▇▇▇▇▇▇████████████████████████████
eval/loss,█▆▅▅▅▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/op-accuracy,▂▁▂▂▄▅▅▇▇▇▇▆▇▇▇▇▇█▇▆▇▇███▇▇▇█▇▇▇▇▇▇███▇█
train/accuracy,▁▁▁▁▂▃▄▄▄▅▇▇▇▇▇█████████████████████████
train/loss,█▅▅▃▃▃▃▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/lr,▁▁▂▂▃▄▄▄▅▅▆▆▇▇▇█████▇▇▇▆▆▆▅▅▅▅▄▄▃▃▃▃▂▂▁▁
train/op-accuracy,▄▃▂▃▁▂▂▄▂▅▄▃▃▅▄▃▃▇▃▇█▃▄▄▃▅▄▄▄▅▆▄▃▆▆▄▅▄▂▅
eval/accuracy,0.93837
eval/loss,0.1641
eval/op-accuracy,0.11938
train/accuracy,0.93085


Start training...


...

  0%|          | 0/6000 [00:00<?, ?it/s]

  0%|          | 0/6000 [00:00<?, ?it/s]

  0%|          | 0/6000 [00:00<?, ?it/s]

  0%|          | 0/6000 [00:00<?, ?it/s]

Training complete.


In [35]:
batch, mask = sampler.generate(1)
input_batch = sampler.decode(batch)

In [55]:
input_batch = ["1+2+3+4=3+3+4"]
batch, mask = sampler.encode(input_batch)

In [56]:
mask.shape

torch.Size([1, 83])

In [57]:
batch = batch.to(config.device)
logits, _ = model(batch)
(logits[:, :-1].argmax(dim=-1) - batch[:, 1:])[mask[:,1:]>0]

tensor([ 0,  0, -2,  1,  3], device='cuda:0')

In [58]:
sampler.decode(logits[:, :-1].cpu().argmax(dim=-1)[mask[:,1:]>0])

['3+1-7']